In [1]:
from SigProfilerAssignment import Analyzer as Analyze
import sigProfilerPlotting as sigPlt
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt 
import seaborn as sns

# COSMIC DATABASE (SigProfilerAssignment)

In [5]:
input_data = '/data/project/Meningioma/41.Signature/01.SigProfiler/Tumor/01.vcf'
output_dir_assignment = '/data/project/Meningioma/41.Signature/01.SigProfiler/Tumor/02.result_cosmic'
output_dir_extractor = '/data/project/Meningioma/41.Signature/01.SigProfiler/Tumor/02.result_extract'
# output_dir_assignment = '/data/project/Meningioma/41.Signature/01.SigProfiler/Tumor/02.assignment'
# output_dir_extractor = '/data/project/Meningioma/41.Signature/01.SigProfiler/Tumor/03.extractor'

if not os.path.exists(output_dir_assignment):
        os.makedirs(output_dir_assignment)

Analyze.cosmic_fit(input_data, output_dir_assignment, input_type = "vcf", genome_build="GRCh38", context_type = "96", exome =True,
                   signatures = None, signature_database = None, collapse_to_SBS96=False, make_plots = True,
                    )  # "Artifact_signatures", exclude_signature_subgroups = ["Chemotherapy_signatures", "Treatment_signatures", "Test2"]

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.61 seconds.
Starting matrix generation for INDELs...Completed! Elapsed time: 3.38 seconds.
Matrices generated for 10 samples with 1482 errors. Total of 11849 SNVs, 216 DINUCs, and 2492 INDELs were successfully analyzed.
Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 10/10 [100%] in 2.7s (3.74/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 


## SBS signature fitting stack bar

In [7]:
f = open(output_dir_assignment  + '/Assignment_Solution/Activities/Assignment_Solution_Activities.txt','r')

data = f.readlines()

column_name = data[0].rstrip().split('\t')
data_split = [x.rstrip().split('\t') for x in data[1:]]

df = pd.DataFrame(data_split, columns = column_name)
df = df.set_index(keys = 'Samples')

for i in df.columns:
    df[i] = pd.to_numeric(df[i])

# 0으로 가득찬 df는 filter하기
for column in df.columns:
    check = set()
    for value in df[column]:
        check.add(value)
    if len(check) == 1:
        df = df.drop(column, axis = 'columns')

df

,SBS1,SBS5,SBS12,SBS15,SBS19,SBS22,SBS28,SBS37,SBS39,SBS42,SBS43,SBS54,SBS55,SBS57,SBS92
Samples,,,,,,,,,,,,,,,
220930_Tumor,7,505,0,0,0,0,0,0,0,0,134,0,0,0,171
221026_Tumor,0,352,0,49,0,0,77,172,0,150,0,0,167,0,0
221102_Tumor,0,0,0,0,0,0,177,0,459,0,0,0,536,0,0
221202_Tumor,0,316,0,0,0,0,0,0,0,0,0,0,778,0,0
230127_Tumor,40,235,177,0,0,0,0,0,195,140,0,0,123,0,0
230323_Tumor,0,752,267,0,0,162,0,0,0,0,0,0,147,203,0
230405_Tumor,0,0,0,0,0,0,145,0,442,0,0,0,497,0,0
230419_Tumor,14,724,373,0,112,0,0,0,256,0,0,148,150,270,0
230526_Tumor,0,264,0,0,0,0,0,0,0,0,0,0,490,0,0


### Percentage 로 변환하기

In [5]:
SBS_name = df.columns
total_count = {}

for index, row in df.iterrows():  ## index == A.~~ row = value
    total = 0
    for value in row:
        total += value
    total_count[index] = total

for index, row in df.iterrows():
    for idx, value in enumerate(row):
        df[SBS_name[idx]][index] = value/total_count[index]*100
        
df

,SBS5,SBS15,SBS16,SBS23,SBS38,SBS39,SBS44,SBS84
Samples,,,,,,,,
220930_Dura,0,55,0,44,0,0,0,0
220930_multiple,75,0,24,0,0,0,0,0
221026_Dura,49,0,0,0,0,0,20,30
221026_multiple,31,0,0,0,21,47,0,0
221102_multiple,0,55,0,44,0,0,0,0


In [44]:
fig, ax = plt.subplots( figsize=(12, 6), nrows = 1, ncols = 1 )
#plt.figure ( figsize = (20, 15) )
sns.set_style("white")
ax.set_title ("Stacked bar chart", fontsize = 24, fontweight='heavy')
df.plot (kind = "bar", stacked=True , ax = ax)
ax.set_xticklabels (list ( df.index ), rotation  = 0, fontsize = 12, fontweight='bold')
ax.set_xlabel("")
plt.savefig ("/home/goldpm1/Meningioma/script/41.SigProfiler/barchart.jpg", dpi = 300)
plt.show()

# DE NOVO SIGNATURE (SigProfilerExtractor)

In [9]:
from SigProfilerExtractor import sigpro as sig

input_data = '/data/project/Meningioma/41.Signature/01.SigProfiler/Dura/01.vcf'
output_dir_extractor = '/data/project/Meningioma/41.Signature/01.SigProfiler/Dura/03.extractor'

sig.sigProfilerExtractor (input_type = "vcf", output = output_dir_extractor, input_data = input_data, reference_genome="GRCh38", context_type = "96,DINUC,ID", exome =True,
                                    minimum_signatures=2, maximum_signatures=5, nmf_replicates=100, resample = True, batch_size=1, cpu=-1, gpu=False, 
                                    nmf_init="random", precision= "single", matrix_normalization= "gmm", seeds = "random", 
                                    min_nmf_iterations= 10000, max_nmf_iterations=1000000, nmf_test_conv= 10000, nmf_tolerance= 1e-15, get_all_signature_matrices= False,
                                    make_decomposition_plots = True)
                   #signatures = None, signature_database = None, collapse_to_SBS96=False, make_plots = True )


************** Reported Current Memory Use: 2.43 GB *****************

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.96 seconds.
Starting matrix generation for INDELs...Completed! Elapsed time: 2.22 seconds.
Matrices generated for 9 samples with 54 errors. Total of 1833 SNVs, 4 DINUCs, and 19 INDELs were successfully analyzed.
Extracting signature 2 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 2 signatures is 22.29 seconds
Optimization time is 3.966200590133667 seconds
The reconstruction error is 0.028, average process stability is 0.98 and 
the minimum process stability is 0.95 for 2 signatures


Extracting signature 3 for mutation type 96
The matrix normalizing cutoff is 9600


Time taken to collect 100 iterations for 3 signatures is 34.24 seconds
Optimization time is 4.119777202606201 seconds
The reconstruction error is 0.195, average process stability is 0.47 and 
the minimum process stability i

### cosine similarity

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

COSMIC_PATH = "/home/goldpm1/resources/COSMIC_SBS/COSMIC_v3.3.1_SBS_GRCh38.txt"
SIGPROFILER_DENOVO_DIR="/data/project/Meningioma/41.Signature/01.SigProfiler/Dura/03.extractor/SBS96/All_Solutions/SBS96_3_Signatures"
SIGPROFILER_DENOVO_TABLE_PATH = SIGPROFILER_DENOVO_DIR + "/Signatures/SBS96_S3_Signatures.txt"

COSMIC_df = pd.read_csv (COSMIC_PATH, sep = "\t")
SIGPROFILER_DENOVO_TABLE_df = pd.read_csv (SIGPROFILER_DENOVO_TABLE_PATH, sep = "\t")



In [39]:
# Compute cosine similarity
cos_sim = cosine_similarity( COSMIC_df.iloc[:, 1:COSMIC_df.shape[1]].values.transpose(), SIGPROFILER_DENOVO_TABLE_df.iloc[:,1:SIGPROFILER_DENOVO_TABLE_df.shape[1]].values.transpose()  )


# 가장 비슷한 (= cosine similarity가 가장 높은 SBS) SBS 찾기
for signature_i in range ( cos_sim.shape[1] ):
    lst = sorted(  cos_sim[:, signature_i], reverse=True)
    lst_arg = []
    for i in lst:
        lst_arg.append ( list(cos_sim[:, signature_i]).index (i) )

    max = np.max ( cos_sim[:, signature_i] )
    max_arg = np.argmax ( cos_sim[:, signature_i] )
    #print ("S{}\tmax = {}\tmax_arg = {}".format(signature_i + 1, round (max, 2), COSMIC_df.columns[ max_arg ] )) 
    print ("SBS96{}\tlst[0:5] = {}\t\tlst_arg[0:5] = {}".format ( chr(signature_i + 65), np.round (lst[0:5], 2) ,  COSMIC_df.columns [ [i for i in  lst_arg [0:5]] ]  )  )

SBS96A	lst[0:5] = [0.91 0.76 0.67 0.59 0.56]		lst_arg[0:5] = Index(['SBS48', 'SBS44', 'SBS37', 'SBS3', 'SBS52'], dtype='object')
SBS96B	lst[0:5] = [0.77 0.63 0.62 0.6  0.57]		lst_arg[0:5] = Index(['SBS93', 'SBS41', 'SBS44', 'SBS3', 'SBS37'], dtype='object')
SBS96C	lst[0:5] = [0.8  0.72 0.63 0.54 0.47]		lst_arg[0:5] = Index(['SBS14', 'SBS5', 'SBS41', 'SBS43', 'SBS22'], dtype='object')


In [40]:
COSMIC_df.iloc[:, 1:COSMIC_df.shape[1]].values.transpose().shape

(79, 96)